In [ ]:
## RUn in the main directory -> same as the main.py file

import numpy as np
import torch
from utils import load
from matplotlib import pyplot as plt
from scipy.spatial import distance
import seaborn as sns
import sklearn
import pickle as pkl
import os

In [ ]:
modularity = [4,1,2,16]

bt = [8,12,16,20,24,36,48]
batch_size = [16,8,8,8,8,8,8,8]
epochs = [120,60,40,40,40,30,20]
lr = [0.1,0.1,0.1,0.1,0.1,0.1,0.1]
seed = [0,1,2,5]
bn = 1.0
scheduler = 'exp'
experiment = 'train'
dev = load.load_device('0')

In [ ]:
e = []
e2 = []
for s in seed:
    E = []
    E2 = []
    lkj = 0
    for b in bt:
        if b !=36 and b!=48:
            arch = [4]
            arch.append(b)
        else:
            arch = [4]
            arch.append(b)
        arch.append(b)
        arch.append(16)
        direct = './results/exemplar/dense/' + str(modularity) + '/' + str(arch) + '/' 
        net = load.load_model(arch,False)(arch, 'relu', False).to(dev)
        net.load_state_dict(torch.load(direct + experiment + str(modularity) + str(lr[lkj]) + str(batch_size[lkj]) + str(epochs[lkj]) + str(arch)  + str(bn) + '_Model' + str(s) + '.pth'))

        i = 0
        start_layer = 2
        start_layer2 = 1
        for p in net.parameters():
            if len(p.data.size())!=1:
                if i < start_layer:
                    if i==0:
                        ewp1 = np.transpose(p.data.abs().detach().cpu().numpy())
                    else:
                        ewp1 = np.matmul(ewp1, np.transpose(p.data.abs().detach().cpu().numpy()))
                    
                if i>=start_layer:
                    if i==start_layer:
                        ewp2 = np.transpose(p.data.abs().detach().cpu().numpy())
                    else:
                        ewp2 = np.matmul(ewp2, np.transpose(p.data.abs().detach().cpu().numpy()))
                if i>=start_layer2:
                    if i==start_layer2:
                        ewp3 = np.transpose(p.data.abs().detach().cpu().numpy())
                    else:
                        ewp3 = np.matmul(ewp3, np.transpose(p.data.abs().detach().cpu().numpy()))
                i+=1
        ewp1 = np.transpose(ewp1)
        ewp1 = np.sum(ewp1, axis=1)
        ewp2 = np.sum(ewp2, axis=1)
        ewp3 = np.sum(ewp3, axis=1)
        E.append(ewp2)
        E2.append(ewp3)
        lkj+=1
    e.append(E)
    e2.append(E2)

In [ ]:
count = []
for lkj in range(len(seed)):
    count.append([])
    E = e[lkj]
    for i in range(len(E)):
        p = np.flip(np.sort(E[i]))
        k = np.sum(p)*0.9
        s = 0
        c = 0
        for j in range(len(p)):
            s+=p[j]
            if s<=k:
                c+=1
        count[lkj].append(c+1)
print(count)
k = np.mean(count, axis=0)
v = np.std(count, axis=0)
print(k,v)

count2 = []
for lkj in range(len(seed)):
    count2.append([])
    E2 = e2[lkj]
    for i in range(len(E2)):
        p = np.flip(np.sort(E2[i]))
        k2 = np.sum(p)*0.9
        s = 0
        c = 0
        for j in range(len(p)):
            s+=p[j]
            if s<=k2:
                c+=1
        count2[lkj].append(c+1)
k2 = np.mean(count2, axis=0)
v2 = np.std(count2, axis=0)


In [ ]:
fig, ax = plt.subplots()
ax.bar(np.arange(1,len(bt)+1)- 0.2, k2, yerr=v2, align='center', alpha=0.8,width= 0.4,ecolor='black', capsize=10, label='Hidden Layer 1')
ax.bar(np.arange(1,len(bt)+1)+ 0.2, k, yerr=v, align='center', alpha=0.8,width= 0.4,ecolor='black', capsize=10, label='Hidden Layer 2')

ax.grid(True)
plt.xticks(np.arange(1,len(bt)+1), bt)

plt.suptitle('Number of hidden units that form \n90% of edge weight product', fontsize=13)
plt.ylabel('Number of units', fontsize=13)
plt.xlabel('NN width', fontsize=13)
plt.legend(fontsize=12)
plt.savefig('ewp_hu_op.png',facecolor='white', bbox_inches='tight')

In [ ]:
e = []
e2 = []

for s in seed: 
    E = []
    E2 = []
    lkj = 0
    for b in bt:
        if b !=36 and b!=48:
            arch = [4]
            arch.append(b)
        else:
            arch = [4]
            arch.append(b)
        arch.append(b)
        arch.append(16)
        pg = [5.0,2.5,2.0,1.5,1.0,0.5]
        fg = [1.0]


        direct_models = './results/exemplar/edge/'  + str(modularity) + '/Models/' +str(arch) + '/' 
        direct_results = './results/exemplar/edge/'  + str(modularity) + '/Results/' +str(arch) + '/' 
        direct_masks = './results/exemplar/edge/'  + str(modularity) + '/Masks/' +str(arch) + '/' 

        pp = []
        ff = []
        ppg = []
        for i in range(len(pg)):
            for j in range(len(fg)):
                with open(direct_results+ 'prune_edgesboolean' + str(lr[lkj]) + str(batch_size[lkj])+ str(epochs[lkj]) + str(None) + str(pg[i]) + str(None) + str(fg[j]) +'_Results' + str(s) +'.pkl', "rb") as fout:
                    re = pkl.load(fout)
                pp.append(re[-1])
                ff.append(fg[j])
                ppg.append(pg[i])
    

        pruning_gradient = ppg[np.argmax(pp)]
        fn_factor = ff[np.argmax(pp)]
        prune_perc = np.max(pp)

        model = load.load_model(arch,False)(arch, 'relu', False).to(dev)
        model.load_state_dict(torch.load(direct_models + 'prune_edgesboolean' + str(lr[lkj]) + str(batch_size[lkj]) + str(epochs[lkj]) + str(None) + str(pruning_gradient) + str(None) + str(fn_factor) +'_Model' + str(s) +'.pth'))

        with open(direct_masks + 'prune_edgesboolean' + str(lr[lkj]) + str(batch_size[lkj])+ str(epochs[lkj]) + str(None) + str(pruning_gradient) + str(None) + str(fn_factor) +'_Mask' + str(s) +'.pkl', "rb") as fout:
            weight_masks = pkl.load(fout)

        i = 0
        start_layer = 2
        start_layer2 = 1
        for p in model.parameters():
            if len(p.data.size())!=1:
                if i < start_layer:
                    if i==0:
                        ewp1 = np.transpose(p.data.abs().detach().cpu().numpy())
                    else:
                        ewp1 = np.matmul(ewp1, np.transpose(p.data.abs().detach().cpu().numpy()))
                    
                elif i>=start_layer:
                    if i==start_layer:
                        ewp2 = np.transpose(p.data.abs().detach().cpu().numpy())
                    else:
                        ewp2 = np.matmul(ewp2, np.transpose(p.data.abs().detach().cpu().numpy()))
                if i>=start_layer2:
                    if i==start_layer2:
                        ewp3 = np.transpose(p.data.abs().detach().cpu().numpy())
                    else:
                        ewp3 = np.matmul(ewp3, np.transpose(p.data.abs().detach().cpu().numpy()))
                i+=1
        ewp1 = np.transpose(ewp1)
        ewp1 = np.sum(ewp1, axis=1)
        ewp2 = np.sum(ewp2, axis=1)
        ewp3 = np.sum(ewp3, axis=1)
        lkj+=1
        E.append(ewp2)
        E2.append(ewp3)
    e.append(E)
    e2.append(E2)

In [ ]:
count = []
for lkj in range(len(seed)):
    count.append([])
    E = e[lkj]
    for i in range(len(E)):
        p = np.flip(np.sort(E[i]))
        k = np.sum(p)*0.9
        s = 0
        c = 0
        for j in range(len(p)):
            s+=p[j]
            if s<=k:
                c+=1
        count[lkj].append(c+1)
k = np.mean(count, axis=0)
v = np.std(count, axis=0)

count2 = []
for lkj in range(len(seed)):
    count2.append([])
    E2 = e2[lkj]
    for i in range(len(E2)):
        p = np.flip(np.sort(E2[i]))
        k2 = np.sum(p)*0.9
        s = 0
        c = 0
        for j in range(len(p)):
            s+=p[j]
            if s<=k2:
                c+=1
        count2[lkj].append(c+1)
k2 = np.mean(count2, axis=0)
v2 = np.std(count2, axis=0)


In [ ]:
fig, ax = plt.subplots()
ax.bar(np.arange(1,len(bt)+1)- 0.2, k2, yerr=v2, align='center', alpha=0.8,width= 0.4,ecolor='black', capsize=10, label='Hidden Layer 1')
ax.bar(np.arange(1,len(bt)+1)+ 0.2, k, yerr=v, align='center', alpha=0.8,width= 0.4,ecolor='black', capsize=10, label='Hidden Layer 2')
ax.grid(True)
plt.xticks(np.arange(1,len(bt)+1), bt)

plt.suptitle('Number of hidden units in edge pruned NN that \nform 90% of edge weight product', fontsize=13)
plt.ylabel('Number of units', fontsize=13)
plt.xlabel('NN width', fontsize=13)
plt.legend(fontsize=12)
plt.savefig('ewp_hu_op_edge.png',facecolor='white', bbox_inches='tight')

In [ ]:
e = []
e2 = []
seed = [0,1,2,5]
for s in seed: 
    E = []
    E2 = []
    lkj = 0
    for b in bt:
        if b !=36 and b!=48:
            arch = [4]
            arch.append(b)
        else:
            arch = [4]
            arch.append(b)
        arch.append(b)
        arch.append(16)
        

        direct_models = './results/exemplar/unit_edge/'  + str(modularity) + '/Models/' +str(arch) + '/' 
        direct_results = './results/exemplar/unit_edge/'  + str(modularity) + '/Results/' +str(arch) + '/' 
        direct_masks = './results/exemplar/unit_edge/'+ str(modularity) + '/Masks/' +str(arch) + '/' 
        pg = [2.5,2.0,1.5,1.0,0.5]
        fg = 1.0
        gradients = [70.0,65.0,60.0,55.0,50.0,45.0,40.0,35.0,30.0,25.0,20.0,15.0,10.0,5.0]

        pp = []
        ff = []
        ppg = []
        for i in range(len(pg)):
            for j in range(len(gradients)):
                path = direct_results + 'pruneactivation_gradient_productboolean' + str(lr[lkj]) + str(batch_size[lkj]) + str(epochs[lkj])  + str(gradients[j]) + str(pg[i]) + str(None) + str(fg) +'_Results'+ str(s) + '.pkl'
                if os.path.isfile(path) == True:
                    with open(direct_results + 'pruneactivation_gradient_productboolean' + str(lr[lkj]) + str(batch_size[lkj]) + str(epochs[lkj])  + str(gradients[j]) + str(pg[i]) + str(None) + str(fg) +'_Results'+ str(s) + '.pkl', "rb") as fout:
                        re = pkl.load(fout)
                    pp.append(re[-1])
                    ff.append(gradients[j])
                    ppg.append(pg[i])

        pruning_gradient = ppg[np.argmax(pp)]
        unit_pruning_gradient = ff[np.argmax(pp)]
        fn_factor = fg
        prune_perc = np.max(pp)

        model = load.load_model(arch,False)(arch, 'relu', False).to(dev)
        model.load_state_dict(torch.load(direct_models + 'pruneactivation_gradient_productboolean' + str(lr[lkj]) + str(batch_size[lkj]) + str(epochs[lkj]) + str(unit_pruning_gradient) + str(pruning_gradient) + str(None) + str(fg) +'_Model' + str(s) + '.pth'))

        with open(direct_masks + 'pruneactivation_gradient_productboolean' + str(lr[lkj]) + str(batch_size[lkj]) + str(epochs[lkj]) + str(unit_pruning_gradient) + str(pruning_gradient) + str(None) + str(fg) +'_Mask'+ str(s) + '.pkl', "rb") as fout:
            weight_masks = pkl.load(fout)

        i = 0
        start_layer = 2
        for p in model.parameters():
            if len(p.data.size())!=1:
                if i < start_layer:
                    if i==0:
                        ewp1 = np.transpose(p.data.abs().detach().cpu().numpy())
                    else:
                        ewp1 = np.matmul(ewp1, np.transpose(p.data.abs().detach().cpu().numpy()))
                    
                elif i>=start_layer:
                    if i==start_layer:
                        ewp2 = np.transpose(p.data.abs().detach().cpu().numpy())
                    else:
                        ewp2 = np.matmul(ewp2, np.transpose(p.data.abs().detach().cpu().numpy()))
                if i>=start_layer2:
                    if i==start_layer2:
                        ewp3 = np.transpose(p.data.abs().detach().cpu().numpy())
                    else:
                        ewp3 = np.matmul(ewp3, np.transpose(p.data.abs().detach().cpu().numpy()))
                i+=1
        ewp1 = np.transpose(ewp1)
        ewp1 = np.sum(ewp1, axis=1)
        ewp2 = np.sum(ewp2, axis=1)
        ewp3 = np.sum(ewp3, axis=1)
        lkj+=1
        E.append(ewp2)
        E2.append(ewp3)
    e.append(E)
    e2.append(E2)

In [ ]:
count = []
for lkj in range(len(seed)):
    count.append([])
    E = e[lkj]
    for i in range(len(E)):
        p = np.flip(np.sort(E[i]))
        k = np.sum(p)*0.9
        s = 0
        c = 0
        for j in range(len(p)):
            s+=p[j]
            if s<=k:
                c+=1
        count[lkj].append(c+1)
k = np.mean(count, axis=0)
v = np.std(count, axis=0)

count2 = []
for lkj in range(len(seed)):
    count2.append([])
    E2 = e2[lkj]
    for i in range(len(E2)):
        p = np.flip(np.sort(E2[i]))
        k2 = np.sum(p)*0.9
        s = 0
        c = 0
        for j in range(len(p)):
            s+=p[j]
            if s<=k2:
                c+=1
        count2[lkj].append(c+1)
k2 = np.mean(count2, axis=0)
v2 = np.std(count2, axis=0)


In [ ]:
fig, ax = plt.subplots()
ax.bar(np.arange(1,len(bt)+1)- 0.2, k2, yerr=v2, align='center', alpha=0.8,width= 0.4,ecolor='black', capsize=10, label='Hidden Layer 1')
ax.bar(np.arange(1,len(bt)+1)+ 0.2, k, yerr=v, align='center', alpha=0.8,width= 0.4,ecolor='black', capsize=10, label='Hidden Layer 2')
ax.grid(True)
plt.xticks(np.arange(1,len(bt)+1), bt)

plt.suptitle('Number of hidden units in unit-edge pruned NN\n that form 90% of edge weight product', fontsize=13)
plt.ylabel('Number of units', fontsize=13)
plt.xlabel('NN width', fontsize=13)
plt.legend(fontsize=12)
plt.savefig('ewp_hu_op_unit_edge.png',facecolor='white', bbox_inches='tight')